In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../input/Combined_News_DJIA.csv")

I made this a binary classification task. Hence, there are only two labels:

"1" when DJIA Adj Close value rose or stayed as the same;

"0" when DJIA Adj Close value decreased.

-

We want +1 and -1 as labels:


In [3]:
data = data.fillna("")
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [4]:
new_data = {'Date': [],'Label': [],'News':[]}

In [5]:
import requests, bs4
def search(word):
    """
    Search for a word 'word' in duden
    """
    url = 'http://www.duden.de/suchen/dudenonline/{word}'.format(word=word)
    response_text = requests.get(url).text
    soup = bs4.BeautifulSoup(response_text, 'html.parser')
    definitions = soup.find_all('h2', class_='vignette__title')
    if len(definitions)<1:
        return None
    
    return  min(definitions, key=len).text



In [6]:
from datetime import datetime
import nltk
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize 
import string
from nltk.corpus import stopwords  

from textblob import TextBlob, Word

stop_words = set(stopwords.words('english')) 
# ps = SnowballStemmer("english") 
ps = nltk.wordnet.WordNetLemmatizer()
# ps = PorterStemmer()

from hunspell import Hunspell
h = Hunspell()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from word_forms.lemmatizer import lemmatize

import spacy 
nlp=spacy.load('en_core_web_sm')

for index, row in data.iterrows():
    print(datetime.now().time())
    combined_new = ""
    label = 1
    for column in row.iloc[2:]:
        combined_new+=" "+column[2:-1]
    if row["Label"] == 0:
        label = -1
    combined_new=combined_new.lower()
    combined_new=combined_new.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(combined_new)
   
    stemmed_words = []

    for word in words:
        try:
#             stemmed_word = h.stem(word)[-1]
#             stemmed_word = wordnet_lemmatizer.lemmatize(breaking)
            stemmed_word = lemmatize(word)
#             stemmed_word = Word(word).lemmatize()
#             stemmed_word = nlp(word)[0].lemma_
#             search_word = search(word).strip()
            if len(stemmed_word) == len(word):
                stemmed_word = word
        except:
            stemmed_word = word
        if len(word) > 3 and stemmed_word not in stop_words and word.isalpha():
            stemmed_words.append(stemmed_word)
    new_data["Date"].append(row["Date"])
    new_data["Label"].append(label)
    new_data["News"].append(stemmed_words)

06:48:41.153010
06:48:42.022825
06:48:42.328231
06:48:42.675394
06:48:42.943395
06:48:43.468402
06:48:43.848399
06:48:44.101398
06:48:44.389399
06:48:44.661401
06:48:44.952403
06:48:45.288399
06:48:45.618398
06:48:45.814395
06:48:46.045397
06:48:46.235395
06:48:46.472421
06:48:46.767397
06:48:46.969401
06:48:47.228407
06:48:47.508407
06:48:47.759398
06:48:48.074407
06:48:48.253401
06:48:48.446405
06:48:48.691400
06:48:48.883405
06:48:49.339395
06:48:49.490397
06:48:49.710397
06:48:49.917405
06:48:50.109003
06:48:50.255004
06:48:50.404016
06:48:50.553247
06:48:50.679357
06:48:50.874364
06:48:51.076353
06:48:51.322359
06:48:51.508695
06:48:51.688694
06:48:51.835708
06:48:51.985974
06:48:52.153439
06:48:52.331458
06:48:52.540457
06:48:52.702458
06:48:52.928461
06:48:53.106464
06:48:53.328457
06:48:53.572891
06:48:53.766892
06:48:53.992892
06:48:54.184895
06:48:54.333548
06:48:54.535547
06:48:54.733145
06:48:54.868784
06:48:55.010781
06:48:55.207787
06:48:55.314785
06:48:55.528919
06:48:55

06:50:25.279623
06:50:25.480626
06:50:25.717626
06:50:25.921626
06:50:26.122630
06:50:26.221629
06:50:26.392625
06:50:26.567626
06:50:26.775627
06:50:26.957641
06:50:27.205627
06:50:27.409646
06:50:27.644627
06:50:27.886627
06:50:28.032635
06:50:28.316634
06:50:28.515646
06:50:28.703365
06:50:28.874368
06:50:29.113364
06:50:29.349366
06:50:29.582369
06:50:29.746366
06:50:29.907366
06:50:30.113365
06:50:30.386366
06:50:30.604371
06:50:30.760366
06:50:30.887368
06:50:31.043364
06:50:31.246363
06:50:31.482368
06:50:31.694364
06:50:31.985364
06:50:32.189364
06:50:32.393365
06:50:32.712371
06:50:32.957348
06:50:33.201349
06:50:33.439368
06:50:33.671347
06:50:33.897370
06:50:34.116365
06:50:34.351365
06:50:34.553374
06:50:34.750376
06:50:34.913365
06:50:35.119366
06:50:35.300371
06:50:35.459367
06:50:35.700348
06:50:35.906364
06:50:36.215366
06:50:36.421370
06:50:36.647367
06:50:36.875364
06:50:37.103369
06:50:37.320367
06:50:37.587386
06:50:37.890379
06:50:38.157374
06:50:38.377373
06:50:38

06:52:14.053095
06:52:14.279089
06:52:14.482092
06:52:14.685089
06:52:14.877089
06:52:15.077086
06:52:15.318104
06:52:15.482103
06:52:15.666106
06:52:15.823091
06:52:15.945091
06:52:16.107090
06:52:16.320115
06:52:16.470093
06:52:16.673104
06:52:16.851091
06:52:17.071105
06:52:17.250656
06:52:17.426671
06:52:17.653677
06:52:17.885687
06:52:18.178676
06:52:18.452655
06:52:18.689653
06:52:18.873658
06:52:19.121669
06:52:19.312656
06:52:19.554671
06:52:19.739653
06:52:19.971654
06:52:20.146149
06:52:20.389244
06:52:20.616259
06:52:20.832258
06:52:21.038246
06:52:21.192243
06:52:21.421243
06:52:21.564245
06:52:21.746257
06:52:21.935244
06:52:22.205245
06:52:22.403250
06:52:22.614248
06:52:22.812252
06:52:23.074249
06:52:23.337251
06:52:23.592267
06:52:23.790248
06:52:24.064243
06:52:24.360247
06:52:24.641244
06:52:24.851246
06:52:25.028258
06:52:25.288250
06:52:25.556262
06:52:25.760259
06:52:25.957240
06:52:26.164484
06:52:26.425467
06:52:26.674464
06:52:26.873487
06:52:27.124465
06:52:27

06:54:04.773261
06:54:04.935262
06:54:05.092272
06:54:05.338261
06:54:05.490261
06:54:05.648260
06:54:05.806264
06:54:06.011266
06:54:06.187265
06:54:06.456261
06:54:06.658262
06:54:06.902265
06:54:07.083245
06:54:07.285527
06:54:07.470750
06:54:07.712863
06:54:07.931850
06:54:08.173857
06:54:08.408858
06:54:08.586847
06:54:08.846864
06:54:09.048889
06:54:09.238889
06:54:09.408891
06:54:09.624887
06:54:09.825885
06:54:10.024885
06:54:10.231887
06:54:10.395891
06:54:10.568884
06:54:10.785890
06:54:10.959903
06:54:11.195886
06:54:11.334612
06:54:11.527609
06:54:11.705611
06:54:11.904613
06:54:12.100610
06:54:12.271626
06:54:12.467626
06:54:12.643626
06:54:12.858625
06:54:13.117624
06:54:13.294617
06:54:13.518610
06:54:13.734612
06:54:13.951628
06:54:14.174613
06:54:14.366626
06:54:14.577629
06:54:14.789627
06:54:15.030610
06:54:15.250610
06:54:15.440607
06:54:15.729624
06:54:15.961609
06:54:16.155610
06:54:16.351610
06:54:16.552628
06:54:16.773608
06:54:16.986610
06:54:17.217625
06:54:17

In [7]:
len(new_data["Date"])

1989

In [8]:
stemmed_words

['woman',
 'mexico',
 'city',
 'final',
 'receive',
 'birth',
 'certify',
 'die',
 'hour',
 'late',
 'trinidad',
 'alvarez',
 'lira',
 'wait',
 'year',
 'proof',
 'born',
 'chief',
 'back',
 'athens',
 'permanent',
 'olympic',
 'preside',
 'france',
 'say',
 'brexit',
 'donald',
 'trum',
 'itish',
 'must',
 'give',
 'police',
 'hour',
 'notice',
 'threaten',
 'hunger',
 'strike',
 'subject',
 'sex',
 'risk',
 'order',
 'despite',
 'never',
 'convict',
 'crime',
 'nobel',
 'laureate',
 'urge',
 'greenpeace',
 'stop',
 'oppose',
 'azil',
 'huge',
 'spike',
 'numbs',
 'police',
 'kill',
 'ahead',
 'olympic',
 'stria',
 'high',
 'court',
 'annul',
 'preside',
 'elect',
 'narrow',
 'lost',
 'rightwing',
 'candidate',
 'cebook',
 'win',
 'privacy',
 'case',
 'track',
 'belgian',
 'want',
 'doesnt',
 'matter',
 'internet',
 'use',
 'log',
 'facebook',
 'itzerland',
 'deny',
 'muslim',
 'girl',
 'citizen',
 'refuse',
 'swim',
 'boy',
 'school',
 'long',
 'consider',
 'naturalise',
 'citizen',


In [9]:
df = pd.DataFrame(new_data, columns = ['Date', 'Label','News'])
len(df)

1989

In [10]:
df.head()

,Date,Label,News
0,2008-08-08,-1,"[georgia, russian, warplane, country, move, br..."
1,2008-08-11,1,"[wont, america, nato, help, wont, help, help, ..."
2,2008-08-12,-1,"[remember, adore, sang, open, ceremony, fake, ..."
3,2008-08-13,-1,"[refuse, israel, weapon, attack, iran, report,..."
4,2008-08-14,1,"[expert, admit, legalise, drug, south, osetia,..."


Check if theres no 0 in data

In [11]:
data.loc[0]["Top1"]

'b"Georgia \'downs two Russian warplanes\' as countries move to brink of war"'

In [12]:
df.loc[0]["News"]

['georgia',
 'russian',
 'warplane',
 'country',
 'move',
 'brink',
 'break',
 'musharraf',
 'impeach',
 'russia',
 'today',
 'column',
 'troop',
 'roll',
 'south',
 'ossetia',
 'footage',
 'fight',
 'youtube',
 'russian',
 'tank',
 'move',
 'towards',
 'capital',
 'south',
 'ossetia',
 'reported',
 'complete',
 'destroy',
 'georgian',
 'artillery',
 'fire',
 'afghan',
 'child',
 'rap',
 'impunity',
 'office',
 'say',
 'sick',
 'three',
 'year',
 'rap',
 'nothing',
 'russian',
 'tank',
 'enter',
 'south',
 'ossetia',
 'whilst',
 'georgia',
 'shot',
 'russian',
 'jet',
 'break',
 'georgia',
 'invade',
 'south',
 'ossetia',
 'russia',
 'warn',
 'would',
 'intervene',
 'side',
 'enemy',
 'combatent',
 'try',
 'nothing',
 'sham',
 'salim',
 'haman',
 'sentence',
 'year',
 'kept',
 'long',
 'anyway',
 'feel',
 'like',
 'georgian',
 'troop',
 'retreat',
 'osettain',
 'capital',
 'presumably',
 'leave',
 'several',
 'hundred',
 'people',
 'kill',
 'video',
 'prep',
 'georgia',
 'russia',
 'ri

In [13]:
df.isnull().sum()

Date     0
Label    0
News     0
dtype: int64

In [14]:
preprocessed_data = df.sample(frac=1).reset_index(drop=False)

In [15]:
# train / testsplit with 80/20%
train_set =preprocessed_data.loc[0:len(preprocessed_data)*0.8] 
test_set = preprocessed_data.loc[len(preprocessed_data)*0.8:] 

In [16]:
train_set.to_pickle("train.pkl")
test_set.to_pickle("test.pkl")

In [17]:
# # DIVIDING THE DATASET INTO TRAINING AND TEST SETS ACCORDING TO DATE
# train= df[df['Date']< '20150101' ]
# test=  df[df['Date']> '20141231']

# # REMOVING PUNCTUATIONS
# data= train.iloc[:,2:27]
# data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# # RENAMING COLUMN NAME FOR EASE OF ACCESS
# list1=[i for i in range(25)]
# new_Index=[str(i) for i in list1]
# data.columns= new_Index
# data.head()

# # CONVERTING THE HEADLINES INTO LOWER CASE
# # SO THAT FOR EXAMPLE, united and United ARE TREATD EQUALLY

# # COMBINING THE TOP 25 HEADLINES OF 1ST RECORD
# ' '.join(str(x) for x in data.iloc[1,0:25])